In [1]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
RAW_DATA_DIR = Path("../data/raw/")

In [7]:
RAW_DATA_DIR


PosixPath('../data/raw')

In [ ]:
df = pd.read_json(RAW_DATA_DIR / "transactions.json", lines=True)

In [ ]:
df.to_parquet(RAW_DATA_DIR / "transactions.parquet")